# Материалы

- [Luciano Ramalho - Decorators and descriptors decoded - PyCon 2017](https://youtu.be/81S01c9zytE)
- https://benmyers.dev/blog/scope/

## Функция - объект

In [1]:
def inc(val: int, delta: int) -> str:
    """My func"""
    x = 10
    return val + delta

In [2]:
inc.__doc__

'My func'

In [3]:
inc.__annotations__

{'val': int, 'delta': int, 'return': str}

In [4]:
inc.__code__.co_varnames

('val', 'delta', 'x')

In [5]:
from inspect import signature
param = signature(inc).parameters['val']
param

<Parameter "val: int">

## Local vs global

In [6]:
b = 1
def func(a):
    print(a)
    print(b)
    b = 3  # assignment in local context => local var

try:
    func(3)
except Exception as e:
    print(e)

3
local variable 'b' referenced before assignment


In [7]:
b = 1
def func(a):
    global b
    print(a)
    print(b)
    b = 3  # assignment in local contex => local var

try:
    func(3)
except Exception as e:
    print(e)

3
1


## Closure

In [8]:
name = 'Ben'

def print_name():
    print(name)

def set_name():
	name = 'Myers'
	print_name()

set_name()  # => `Ben` (NOT `Myers`)

Ben


https://benmyers.dev/blog/scope/

### Lexical scope (= static scope)

`JavaScript` and other languages such as the `C family` and `Python` use **lexical scope**, also called **static scope**, which means that scope nests according to where functions and variables are declared. When they encounter a reference to a variable, lexically scoped languages ask "Where was this written? Where was that written?" and so forth until they find a variable declaration.

In lexically scoped languages, variable references are predictable. For instance, name didn't change what it referred to based on whether logName was invoked in the global scope or inside setName. This predictability comes at the cost of more required overhead, generally handled at compile time.

### Dynamic scope

`Bash`, on the other hand, uses **dynamic scope**, where scope is nested based on the order of execution. Dynamic scope is handled at runtime, and tends to require a little less overhead than lexical scope. It comes at a high cost of unpredictability—the same line of code in a function could refer to two different things depending on where the function was invoked, and subprograms could have the potential to unwittingly overwrite your variables. It's for this reason that the field has largely moved to lexically scoped languages.

### Closures

A function's lexical environment is the set of all variables and functions that have been defined in the scope chain when the function is declared.

This combination of a function and its lexical environment is called a closure.

### Класс производит instance, который ведет себя как функция

In [9]:
class Averager:
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        return sum(self.series)/len(self.series)

avg = Averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

### Функция производит функцию

Вызов `make_averager` -> new instance of averager with own `series` (это обеспечивает independent state).

`series` - free variable


https://stackoverflow.com/questions/12919278/how-to-define-free-variable-in-python

In [10]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = len(series)
        return sum(series)/total
    return averager

avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

In [11]:
avg.__code__.co_varnames

('new_value', 'total')

In [12]:
avg.__code__.co_freevars

('series',)

In [13]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [14]:
avg(13)
avg.__closure__[0].cell_contents

[10, 11, 12, 13]

In [15]:
def outer():
    f = []
    def inner1():
        f.append('1')
        print(f)
    def inner2():
        f.append('10')
        print(f)
    inner1.x = inner2
    return inner1, inner2


In [16]:
in1, in2 = outer()

In [17]:
in1()

['1']


In [18]:
in2()

['1', '10']


In [19]:
in1.x()

['1', '10', '10']


### nonlocal

In [20]:
# Более эффективный способ
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

# avg = make_averager() => UnboundLocalError
# assignment -> means local variable -> count = count (???) + 1 

def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager
